In [1]:
import pandas as pd 
import numpy as np
import requests
import json
import time
import datetime as dt
import re
import os.path
from openpyxl import load_workbook
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
Area_Code = {
    '12':['Algeria','DZA'],
    '24':['Angola','AGO'],
    '204':['Benin','BEN'],
    '72':['Botswana','BWA'],
    '854':['Burkina Faso','BFA'],
    '108':['Burundi','BDI'],
    '132':['Cabo Verde','CPV'],
    '120':['Cameroon','CMR'],
    '140':['Central African Republic','CAF'],
    '148':['Chad','TCD'],
    '174':['Comoros','COM'],
    '178':['Congo','COG'],
    '384':['Côte d’Ivoire','CIV'],
    '180':['Democratic Republic of the Congo','COD'],
    '262':['Djibouti','DJI'],
    '818':['Egypt','EGY'],
    '226':['Equatorial Guinea','GNQ'],
    '232':['Eritrea','ERI'], #232
    '748':['Eswatini','SWZ'],
    '231':['Ethiopia','ETH'],
    '266':['Gabon','GAB'],
    '270':['Gambia','GMB'],
    '288':['Ghana','GHA'],
    '324':['Guinea','GIN'],
    '624':['Guinea-Bissau','GNB'],
    '404':['Kenya','KEN'],
    '426':['Lesotho','LSO'],
    '430':['Liberia','LBR'],
    '434':['Libya','LBY'],
    '450':['Madagascar','MDG'],
    '454':['Malawi','MWI'],
    '466':['Mali','MLI'],
    '478':['Mauritania','MRT'],
    '480':['Mauritius','MUS'],
    '504':['Morocco','MAR'],
    '508':['Mozambique','MOZ'],
    '516':['Namibia','NAM'],
    '562':['Niger','NER'],
    '566':['Nigeria','NGA'],
    '646':['Rwanda','RWA'],
    '686':['Senegal','SEN'],
    '690':['Seychelles','SYC'],
    '694':['Sierra Leone','SLE'],
    '706':['Somalia','SOM'],
    '710':['South Africa','ZAF'],
    '728':['South Sudan','SSD'],
    '678':['São Tomé and Príncipe','STP'],
    '729':['Sudan','SDN'],
    '768':['Togo','TGO'],
    '788':['Tunisia ','TUN'],
    '800':['Uganda','UGA'],
    '834':['United Republic of Tanzania','TZA'], #835 834
    '894':['Zambia','ZMB'],
    '716':['Zimbabwe','ZWE'],
}
select_code = [int(k) for k in Area_Code.keys()]
# Getting year for citation
D = dt.datetime.today()
y = D.year

In [3]:
len(select_code)

54

In [4]:
URL_Dic={
    2:['GDP breakdown at current prices','USD','',],
    6:['GDP breakdown at constant prices','USD','',],
    9:['Per Capita GDP at current prices in US Dollars','USD',''],
    16:['Growth Rate of GDP breakdown at constant 2015 prices','PERC','GGR'],
    22:['Percentage Distribution (Shares) of GDP','PERC',''],
    24:['GNI at current prices in US Dollars','USD',''],
    27:['Per Capita GNI at current prices in US Dollars','USD',''],
}

In [5]:
Indicators = {
    'Agriculture, hunting, forestry, fishing (ISIC A-B)':'ECO.NAC.GDP.ACT.CON.001',
    'Changes in inventories':'ECO.NAC.GDP.CON.006',
    'Construction (ISIC F)':'ECO.NAC.GDP.ACT.CON.005',
    'Education':'ECO.NAC.GDP.ACT.CON.010',
    'Electricity, gas & wate':'ECO.NAC.GDP.ACT.CON.004',
    'Exports of goods and services':'ECO.NAC.GDP.CON.007',
    'Final consumption expenditure':'ECO.NAC.GDP.CON.001',
    'Finance, insurance, real estate, etc.':'ECO.NAC.GDP.ACT.CON.007',
    'General government final consumption expenditure':'ECO.NAC.GDP.CON.003',
    'Gross capital formation':'ECO.NAC.GDP.CON.004',
    'Gross Domestic Product (GDP)':'ECO.NAC.GDP.ACT.CON.016',
    'Gross fixed capital formation (including Acquisitions less disposals of valuables)':'ECO.NAC.GDP.CON.005',
    'Health And Social Work':'ECO.NAC.GDP.ACT.CON.011',
    'Household consumption expenditure (including Non-profit institutions serving households)':'ECO.NAC.GDP.CON.002',
    'Imports of goods and services':'ECO.NAC.GDP.CON.008',
    'Less Imputed Service Charges':'ECO.NAC.GDP.ACT.CON.013',
    'Manufacturing (ISIC D)':'ECO.NAC.GDP.ACT.CON.003',
    'Mining and quarrying':'ECO.NAC.GDP.ACT.CON.002',
    'Mining, Manufacturing, Utilities (ISIC C-E)':'ECO.NAC.GDP.ACT.CON.022',
    'Other Activities (ISIC J-P)':'ECO.NAC.GDP.ACT.CON.099',
    'Other services':'ECO.NAC.GDP.ACT.CON.012',
    'Plus: Indirect Taxes / taxes on products, less subsidies':'ECO.NAC.GDP.ACT.CON.015',
    'Public administration and defense':'ECO.NAC.GDP.ACT.CON.009',
    'Total Value Added':'ECO.NAC.GDP.ACT.CON.999',
    'Transport, storage and communication (ISIC I)':'ECO.NAC.GDP.ACT.CON.008',
    'Wholesale, retail trade, restaurants and hotels (ISIC G-H)':'ECO.NAC.GDP.ACT.CON.006',
    'GNI at current prices in US Dollars':'ECO.NAC.GNI.TOT.001',
    'Per Capita GNI at current prices in US Dollars':'ECO.NAC.GNI.PCAP.001',
    'Per Capita GDP at current prices in US Dollars':'GDP per capita'
}

In [6]:
# Headers for the Data Frame
yearH = list(range(1970,y+1))
cols = ['Areacode','Areaenglish','DataSource','DataProviderId','SpecificSource',\
        'Sector','Indicatorcode','Indicatorenglish','Modecode','Unitcode','BaseYear']
columns = cols.extend(yearH)
# Creating empty Data Frame with the above headers
DataFrame = pd.DataFrame(columns = cols)
# Data Source
DataSource='United Nations Statistics Division'
# Data Provider ID
DataProviderID=9
for num in URL_Dic:
    print(num)
    #target URL
    url = f'https://unstats.un.org/unsd/amaapi/api/file/{num}'
    # make HTTP GET request to the target URL
    response = None
    a = 0
    while response is None:
        try:
            a +=1
            print("atempt:",a)
            response = requests.get(url)
        except:
            time.sleep(3)
            continue
    
    content = response.content
    # writting the scraped file to an excel file.
    csv_file = open(f'{num}.xlsx', 'wb')
    csv_file.write(content)
    
    #Reading the saved excel file
    Data = pd.read_excel(f'{num}.xlsx', sheet_name=0, engine='openpyxl')
    title = Data.columns[0]
    title = title.replace('/',' ')
    if title[-1]==')':
        start = title.rfind('(')
        title = title[:start]
    print(title)
        
    Data.columns = Data.loc[1]
    Data = Data.loc[2:]
    Yh = list(Data.columns[3:].astype(int))
    Yf = list(Data.columns[:3])
    Yf.extend(Yh)
    Data.columns = Yf
    
    sector = URL_Dic[num][0]
    if 'IndicatorName' not in Data.columns:
        Data.insert(2,'IndicatorName', [sector]*len(Data))
    
    Data.insert(2,'Indicatorcode', ['']*len(Data))

    unitcode = URL_Dic[num][1]
    modecode = URL_Dic[num][2]
    Data.insert(2,'Unitcode', [unitcode]*len(Data))
    Data.insert(2,'Modecode', [modecode]*len(Data))   
    
    if 'constant' in title:
        baseYear = int(re.findall('\d+', title )[0])
        Data.insert(2,'BaseYear', [baseYear]*len(Data))
    else:
        Data.insert(2,'BaseYear', ['']*len(Data))

    Data.insert(2,'Sector',[sector]*len(Data))

    Data.rename(columns = {'CountryID':'Areacode','Country':'Areaenglish','IndicatorName':'Indicatorenglish'}, inplace = True)
    Data=Data[Data['Areacode'].isin(select_code)]

    Data = Data.reset_index(drop=True)
    
    for i in range(len(Data)):
        code = str(Data.iloc[i,0])
        Data.iloc[i,0] = Area_Code[code][1]
        Data.iloc[i,1] = Area_Code[code][0]
        Data.iloc[i,6] = Indicators[Data.iloc[i,7]]
        
    cite = f'Retrived from: {url}, On: {str(D.date())}, By web scraping using the python libraries: requests and pandas'
    SpecificSource = [cite]*len(Data)
    Data.insert(2, 'SpecificSource', SpecificSource )
    Data.insert(2, 'DataProviderId', [DataProviderID]*len(Data) )
    Data.insert(2, 'DataSource', [DataSource]*len(Data) )

    DataFrame = pd.concat([DataFrame, Data], ignore_index=True, sort=False)


DataFrame.to_excel('GDP.xlsx', index=False)
print('All Done!')

2
atempt: 1
GDP breakdown at current prices in US Dollars 
6
atempt: 1
GDP breakdown at constant 2015 prices in US Dollars 
9
atempt: 1
Per Capita GDP at current prices in US Dollars 
16
atempt: 1
Growth Rate of GDP breakdown at constant 2015 prices in Percent 
22
atempt: 1
Shares of breakdown of GDP Value Added at current prices in Percent 
24
atempt: 1
GNI at current prices in US Dollars 
27
atempt: 1
Per Capita GNI at current prices in US Dollars 
All Done!


In [22]:
DataFrame

,Areacode,Areaenglish,DataSource,DataProviderId,SpecificSource,Sector,Indicatorcode,Indicatorenglish,Modecode,Unitcode,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,DZA,Algeria,United Nations Statistics Division,9,Retrived from: https://unstats.un.org/unsd/ama...,GDP breakdown at current prices,ECO.NAC.GDP.CON.001,Final consumption expenditure,,USD,...,1.128491e+11,1.200609e+11,1.038546e+11,1.014664e+11,1.046321e+11,1.035761e+11,1.034588e+11,9.904572e+10,NaN,NaN
1,DZA,Algeria,United Nations Statistics Division,9,Retrived from: https://unstats.un.org/unsd/ama...,GDP breakdown at current prices,ECO.NAC.GDP.CON.002,Household consumption expenditure (including N...,,USD,...,7.269622e+10,7.774635e+10,6.806887e+10,6.803551e+10,7.239797e+10,7.348807e+10,7.445431e+10,6.988029e+10,NaN,NaN
2,DZA,Algeria,United Nations Statistics Division,9,Retrived from: https://unstats.un.org/unsd/ama...,GDP breakdown at current prices,ECO.NAC.GDP.CON.003,General government final consumption expenditure,,USD,...,4.015287e+10,4.231460e+10,3.578569e+10,3.343093e+10,3.223408e+10,3.008802e+10,2.900447e+10,2.916543e+10,NaN,NaN
3,DZA,Algeria,United Nations Statistics Division,9,Retrived from: https://unstats.un.org/unsd/ama...,GDP breakdown at current prices,ECO.NAC.GDP.CON.004,Gross capital formation,,USD,...,9.101395e+10,9.739992e+10,8.428539e+10,8.126174e+10,8.257144e+10,8.293396e+10,7.871759e+10,6.291135e+10,NaN,NaN
4,DZA,Algeria,United Nations Statistics Division,9,Retrived from: https://unstats.un.org/unsd/ama...,GDP breakdown at current prices,ECO.NAC.GDP.CON.005,Gross fixed capital formation (including Acqui...,,USD,...,7.170227e+10,8.000460e+10,7.013764e+10,6.893384e+10,6.936783e+10,7.042629e+10,6.622852e+10,5.948013e+10,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3626,TGO,Togo,United Nations Statistics Division,9,Retrived from: https://unstats.un.org/unsd/ama...,Per Capita GNI at current prices in US Dollars,ECO.NAC.GNI.PCAP.001,Per Capita GNI at current prices in US Dollars,,USD,...,7.227217e+02,7.850571e+02,7.524217e+02,8.216018e+02,8.354099e+02,8.874621e+02,8.564047e+02,8.738317e+02,NaN,NaN
3627,TUN,Tunisia,United Nations Statistics Division,9,Retrived from: https://unstats.un.org/unsd/ama...,Per Capita GNI at current prices in US Dollars,ECO.NAC.GNI.PCAP.001,Per Capita GNI at current prices in US Dollars,,USD,...,4.034616e+03,4.151526e+03,3.741831e+03,3.593172e+03,3.373668e+03,3.320930e+03,3.204730e+03,3.230401e+03,NaN,NaN
3628,UGA,Uganda,United Nations Statistics Division,9,Retrived from: https://unstats.un.org/unsd/ama...,Per Capita GNI at current prices in US Dollars,ECO.NAC.GNI.PCAP.001,Per Capita GNI at current prices in US Dollars,,USD,...,6.456515e+02,1.036484e+03,7.548025e+02,7.561281e+02,7.490517e+02,7.748105e+02,8.298226e+02,8.322085e+02,NaN,NaN
3629,ZMB,Zambia,United Nations Statistics Division,9,Retrived from: https://unstats.un.org/unsd/ama...,Per Capita GNI at current prices in US Dollars,ECO.NAC.GNI.PCAP.001,Per Capita GNI at current prices in US Dollars,,USD,...,1.799708e+03,1.661642e+03,1.315864e+03,1.240753e+03,1.486688e+03,1.491758e+03,1.282687e+03,9.682445e+02,NaN,NaN
